<a href="https://colab.research.google.com/github/lwztime/vits/blob/main/sovits4_0_v2%E4%B8%80%E9%94%AE%E8%AE%AD%E7%BB%83_%E6%8E%A8%E7%90%86%E8%84%9A%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

初始项目：https://github.com/innnky/so-vits-svc

修改日期：2023年3月13日

原始Colab 链接<a target="_blank" href="https://colab.research.google.com/drive/19fxpo-ZoL_ShEUeZIZi6Di-YioWrEyhR">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [1]:
# 查看显卡
!nvidia-smi
!pwd

Mon Jun 30 05:20:16 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   55C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [1]:
#@title 克隆的github仓库
!git clone https://github.com/innnky/so-vits-svc.git


Cloning into 'so-vits-svc'...
remote: Enumerating objects: 931, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 931 (delta 3), reused 1 (delta 0), pack-reused 913 (from 1)
Receiving objects: 100% (931/931), 25.41 MiB | 15.56 MiB/s, done.
Resolving deltas: 100% (446/446), done.


In [1]:
#@title 安装依赖
%cd /content/so-vits-svc
!pip install --upgrade pip setuptools numpy numba
#!pip install numpy==1.23.5
!pip install pyworld praat-parselmouth fairseq

/content/so-vits-svc
  Using cached numpy-2.3.1-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (62 kB)
  Using cached pyworld-0.3.5.tar.gz (261 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 134.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
Requested omegaconf<2.1 from https://files.pythonhosted.org/packages/d0/eb/9d63ce09dd8aa85767c65668d5414958ea29648a0eec80a4a7d311ec2684/omegaconf-2.0.6-py3-none-any.whl (from fairseq) has invalid metadata: .* suffix can only be used with `==` or `!=` operators
    PyYAML (>=5.1.*)
            ~~~~~~^
Please use pip<24.1 if you need to use this version.
Requested omegaconf<2.1 from https://files.pythonhosted.org/packages/e5/f6/043b6d255dd6fbf2025110cea35b8

In [4]:
#@title 下载必要模型文件
# 源仓库地址：[contentvec](https://github.com/auspicious3000/contentvec)
# 模型原下载链接：[checkpoint_best_legacy_500.pt](https://ibm.box.com/s/z1wgl1stco8ffooyatzdwsqn2psd9lrr)
# 由于源网盘无法提供http直链，根据mit协议，对模型进行二次分发，提供下载直链
#!wget -P hubert/ https://huggingface.co/innnky/contentvec/resolve/main/checkpoint_best_legacy_500.pt
!wget https://public.boxcloud.com/d/1/b1!jNI-oXEcD9nL_QmmbaZfswubs280MVSQRraZAymktBwoKCHa5U-RrnRD3gxItLgdQ7ja0Ev9J5s73s_O6nxlAbDJ9yBJGWciGlBFEST4txPmFu_cob1b4xS8uLUWdpZXGQz_pt-hQXYG6I4w0oX8YzsoIDkgk1bHY62c4x46TdokH9BdViM_V64arxF2HOfbK73tWvaBMxFLnBorvoqOuyi0fpy6ClOY13xInyDWczfwjBMy0YZaldpS5HlbBJUiIpOHairuP6JTBZJklARYj9I6NIV3PfjdkvDak93s1ba51xC6K7kMnDPpR2SkkzR97P2SDfFkBJ5XqBGPUIvYkKXO09dMyCRAMwmjoL7Rq8noy0XHEb5gyNCiei10Nga33JcBTLzSWaeV5THwslvQ29ocKhLm04ZIpBrdxJ2xeF75iGMYOfT-v_dGR3tA8UQcf56Me4NVklkD7DEjiPopu1fUQEunvpnzDq7rmzJ9_hzDh2vfMDMwr1TQtYjmfb2aBZiF5QqeQs5ljUfAFoP6d8S33ENYfY5uTaRKJ31VMVHb-8unQqFFtCCcrNjl_HIb0T1lgjTBQNlV1Yf2d_C1z_urGIuBMYBdQr3KXWpoF4yA-CgZYvvMc3niJOP6wCEvd1SQfCuuZVQI9aeQE4g92rOslUrfo7OIMVXj8W1MiXIx3IVFZpcsAtgYpwTIwNeDhbukHfMs0QRc2plMfH6P36QBD9oNmd_lolpAI_QZxUHYFa0HYjA2x_fhj6XrzQmFKCLse8KHhSI1feVbEmMZ_kJQ4Z8i9hai8I9eiAuYFrfsZdG3RWZoTYExk5Ci64kUqfKyLYrvtb2jWrQTZNR7JHjpdi-WiYic8mkUwZ01RylgU5Nixz_pWNJsPuWIN5hRR30fUU4v2m4gMwoGqTYqXd8WSRn1Fu0EgHU6DPfZIqWBVw6sFclTc1Xw6fl5hCNjxGcktIfuKB5J9238a4fEM-aGgcumky0L0ITcBYWikLJnGuZBoMe56oMXFc7tAsHqvFFrZXSrb0aRusWbc4M0nT8qlmyYX1eZQCHURzNnJ7juN4gaCorpYwK4sdeoYV3L6DGwifJRY1DHmMEyYtk4x0-m4_WPPansJ_c6_wL-Ru5UxKGG0nHy3Hj9m_XjytdGE7QeWFb79rmOPd4S7Yos-IG5qwAkAFNJe-VknvguB02_PyKcyXL31-J9v4gKJsS1ISeMJ6UKWj4nBrgbJLmDCa3nvml80nJgSJM3BcOLC-3Dg0vEEo6TK1ijYrLOpacehYBCUTRzxGhdXseQ05a46b56_A8cFEUqI-TzwXyR8ifVCUlcA9c14wqLYe0nlB1997BvXLHY68Kfo8cBEiXXUR20xvrZtUAwv4Y0-xFxUOxk2VjX6vUyajxpsn4T4YXRhIMj1KceVAi-gA../download
!mv download checkpoint_best_legacy_500.pt

--2025-06-30 05:25:41--  https://public.boxcloud.com/d/1/b1!jNI-oXEcD9nL_QmmbaZfswubs280MVSQRraZAymktBwoKCHa5U-RrnRD3gxItLgdQ7ja0Ev9J5s73s_O6nxlAbDJ9yBJGWciGlBFEST4txPmFu_cob1b4xS8uLUWdpZXGQz_pt-hQXYG6I4w0oX8YzsoIDkgk1bHY62c4x46TdokH9BdViM_V64arxF2HOfbK73tWvaBMxFLnBorvoqOuyi0fpy6ClOY13xInyDWczfwjBMy0YZaldpS5HlbBJUiIpOHairuP6JTBZJklARYj9I6NIV3PfjdkvDak93s1ba51xC6K7kMnDPpR2SkkzR97P2SDfFkBJ5XqBGPUIvYkKXO09dMyCRAMwmjoL7Rq8noy0XHEb5gyNCiei10Nga33JcBTLzSWaeV5THwslvQ29ocKhLm04ZIpBrdxJ2xeF75iGMYOfT-v_dGR3tA8UQcf56Me4NVklkD7DEjiPopu1fUQEunvpnzDq7rmzJ9_hzDh2vfMDMwr1TQtYjmfb2aBZiF5QqeQs5ljUfAFoP6d8S33ENYfY5uTaRKJ31VMVHb-8unQqFFtCCcrNjl_HIb0T1lgjTBQNlV1Yf2d_C1z_urGIuBMYBdQr3KXWpoF4yA-CgZYvvMc3niJOP6wCEvd1SQfCuuZVQI9aeQE4g92rOslUrfo7OIMVXj8W1MiXIx3IVFZpcsAtgYpwTIwNeDhbukHfMs0QRc2plMfH6P36QBD9oNmd_lolpAI_QZxUHYFa0HYjA2x_fhj6XrzQmFKCLse8KHhSI1feVbEmMZ_kJQ4Z8i9hai8I9eiAuYFrfsZdG3RWZoTYExk5Ci64kUqfKyLYrvtb2jWrQTZNR7JHjpdi-WiYic8mkUwZ01RylgU5Nixz_pWNJsPuWIN5hRR30fUU4v2m4gMwoGqTYqXd8WSRn1Fu0EgHU6DPfZIqWB

In [3]:
cp /content/drive/MyDrive/checkpoint_best_legacy_500.pt /content/so-vits-svc/

In [ ]:
!pwd

/content/so-vits-svc


# 数据集预处理

In [2]:
#@title 加载Google云端硬盘
#@markdown 加载Google云端硬盘
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


该处理脚本可以一次性预处理多个说话人，并且生成多说话人filelists以及对应的配置文件

只需要将你的数据集按照以下文件结构放到dataset_raw目录下


```
dataset_raw
├───speaker0
│   ├───xxx1-xxx1.wav
│   ├───...
│   └───Lxx-0xx8.wav
└───speaker1
    ├───xx2-0xxx2.wav
    ├───...
    └───xxx7-xxx007.wav

```




In [ ]:
#@title 从谷歌云盘加载打包好的数据集进行预处理
#@markdown **sovits4.0的数据集不再需要特定的文件结构，将数据集的所有wav文件放在同一个文件夹下压缩为zip后上传至谷歌云盘即可，该处理脚本可以一次性预处理多个数据集，处理多个数据集时请依次解压每一个数据集**

#@markdown 数据集名称（**人物的英文/拼音名**，与建数据文件夹时统一；不带zip。）
DATASETNAME = "ner"  #@param {type:"string"}
#@markdown 压缩包路径（谷歌盘路径，传到dataset的就不改这个，没有dataset文件夹就新建一个）
ZIP_PATH = "/content/drive/MyDrive/dataset/"  #@param {type:"string"}
ZIP_NAME = ZIP_PATH + DATASETNAME

!unzip -d /content/so-vits-svc/dataset_raw {ZIP_NAME}.zip

Archive:  /content/drive/MyDrive/dataset/ner.zip
   creating: /content/so-vits-svc/dataset_raw/ner/
  inflating: /content/so-vits-svc/dataset_raw/ner/ner0001.wav  
  inflating: /content/so-vits-svc/dataset_raw/ner/ner0002.wav  
  inflating: /content/so-vits-svc/dataset_raw/ner/ner0003.wav  
  inflating: /content/so-vits-svc/dataset_raw/ner/ner0004.wav  
  inflating: /content/so-vits-svc/dataset_raw/ner/ner0005.wav  
  inflating: /content/so-vits-svc/dataset_raw/ner/ner0006.wav  
  inflating: /content/so-vits-svc/dataset_raw/ner/ner0007.wav  
  inflating: /content/so-vits-svc/dataset_raw/ner/ner0008.wav  
  inflating: /content/so-vits-svc/dataset_raw/ner/ner0009.wav  
  inflating: /content/so-vits-svc/dataset_raw/ner/ner0010.wav  
  inflating: /content/so-vits-svc/dataset_raw/ner/ner0011.wav  
  inflating: /content/so-vits-svc/dataset_raw/ner/ner0012.wav  
  inflating: /content/so-vits-svc/dataset_raw/ner/ner0013.wav  
  inflating: /content/so-vits-svc/dataset_raw/ner/ner0014.wav  
  in

In [9]:
#@title 重采样到44100hz
!python resample.py

python3: can't open file '/content/so-vits-svc/resample.py': [Errno 2] No such file or directory


In [ ]:
#@title 划分训练集 生成配置文件
!python preprocess_flist_config.py

100% 1/1 [00:00<00:00, 4359.98it/s]
Writing ./filelists/train.txt
100% 47/47 [00:00<00:00, 791695.94it/s]
Writing ./filelists/val.txt
100% 2/2 [00:00<00:00, 74898.29it/s]
Writing ./filelists/test.txt
100% 2/2 [00:00<00:00, 77672.30it/s]
Writing configs/config.json


In [13]:
!cp checkpoint_best_legacy_500.pt ../drive/MyDrive

In [ ]:
!

In [ ]:
#@title 生成hubert和f0
!python preprocess_hubert_f0.py

In [ ]:
#@title 至此 数据集预处理制作完毕，将数据集和相关文件保存到谷歌云盘的dataset文件夹下，方便下次训练使用
#压缩dataset文件夹
!zip -r dataset.zip /content/so-vits-svc/dataset
#@markdown 自定义备份到谷歌云盘的dataset文件夹下的数据集文件夹名，避免混淆
dataset_name_drive = "ner_preprocessed"  #@param {type:"string"}
DATASET_PATH_DRIVE = "/content/drive/MyDrive/dataset/" + dataset_name_drive
!mkdir -p {DATASET_PATH_DRIVE}

!cp /content/so-vits-svc/dataset.zip "{DATASET_PATH_DRIVE}"
!cp configs/config.json "{DATASET_PATH_DRIVE}"
!cp filelists/train.txt "{DATASET_PATH_DRIVE}"
!cp filelists/val.txt "{DATASET_PATH_DRIVE}"

In [ ]:
#@title 已经预处理过数据集的话，就可以跳过预处理部分 直接从云盘解压处理过的数据 以及配置文件
#@markdown 从谷歌云盘加载预处理过的数据集，文件夹名和你备份的时候输入的一样
back_up_name = "kiritan_preprocessed"  #@param {type:"string"}
BACK_UP_DATASET_PATH = "/content/drive/MyDrive/dataset/" + back_up_name
!unzip {BACK_UP_DATASET_PATH}/dataset.zip -d /
!cp {BACK_UP_DATASET_PATH}/config.json /content/so-vits-svc/configs/config.json
!cp {BACK_UP_DATASET_PATH}/val.txt filelists/val.txt
!cp {BACK_UP_DATASET_PATH}/train.txt filelists/train.txt


# 拷贝云盘上保存的记录点
# !cp /content/drive/MyDrive/G_800.pth logs/48k/
# !cp /content/drive/MyDrive/D_800.pth logs/48k/

# 训练

In [ ]:
#@title  选择是否将训练后的模型保存到谷歌云盘 和 是否使用预模型

#@markdown **将训练后的模型文件保存到谷歌云盘，勾选后，恢复训练时也需要勾选并执行**
Save_to_drive = True #@param {type:"boolean"}
if Save_to_drive:
  !rm -rf /content/so-vits-svc/logs/44k
  !mkdir -p /content/drive/MyDrive/44k
  !ln -s /content/drive/MyDrive/44k /content/so-vits-svc/logs/44k

#@markdown **首次训练下载预模型 之后继续训练则使用自己保存的记录点，无需再下载**

#@markdown **使用预模型，下面打钩自动下载并启用**
pre_pth = True #@param {type:"boolean"}
if pre_pth:
  #!wget -P logs/44k/ https://huggingface.co/innnky/sovits_pretrained/resolve/main/sovits4/G_0.pth
  #!wget -P logs/44k/ https://huggingface.co/innnky/sovits_pretrained/resolve/main/sovits4/D_0.pth

  !wget -P logs/44k/ https://huggingface.co/datasets/seq2193/sovits_pretrained/resolve/main/sovits4.0-v2/G_0.pth
  !wget -P logs/44k/ https://huggingface.co/datasets/seq2193/sovits_pretrained/resolve/main/sovits4.0-v2/D_0.pth


In [ ]:
#@title  开始训练

#@markdown **开始训练**

#@markdown **启用tensorboard可视化数据**
tensorboard_on = True #@param {type:"boolean"}
if tensorboard_on:
  %load_ext tensorboard
  %tensorboard --logdir logs/44k

!python train.py -c configs/config.json -m 44k


In [ ]:
#@title 聚类音色泄漏控制 (选用)

#@markdown # 聚类音色泄漏控制 Train cluster model (Optional)

#@markdown #### 詳情參見 [README.md#cluster-based-timbre-leakage-control](https://github.com/svc-develop-team/so-vits-svc#cluster-based-timbre-leakage-control)

#@markdown 执行python cluster/train_cluster.py ，模型的输出会在 logs/44k/kmeans_10000.pt

!python cluster/train_cluster.py

In [ ]:
#@title 手动将训练后的模型文件备份到谷歌云盘
#@markdown 需要自己查看/content/so-vits-svc/logs/44k/文件夹下模型的文件名，手动修改下方命令末尾的文件名
!mv /content/so-vits-svc/logs/44k/G_1000.pth /content/drive/MyDrive
!mv /content/so-vits-svc/logs/44k/D_1000.pth /content/drive/MyDrive

## 加载歌手模型

In [ ]:
#@title 从huggingface下载模型
#@markdown **注意：4.0与4.0-v2模型完全不通用**
pth_link="" #@param{type:"string"}
config_link="" #@param{type:"string"}
cluster_model_path="" #@param{type:"string"}
!wget -P /content/so-vits-svc/logs/44k {pth_link}
!wget -O /content/so-vits-svc/configs/config.json  {config_link}
!wget -P /content/so-vits-svc/logs/44k {cluster_model_path}

# 推理

In [ ]:
#@markdown 下载示例音频合成 風になる
!gdown --id 1d-npX0KOYW5AE2fh0z1mkX3nFYvyrRii -O /content/so-vits-svc/raw/風になる.wav

In [ ]:
#@title 上传.wav音频文件
import os
from google.colab import files

save_dir = "/content/so-vits-svc/raw"
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

uploaded = files.upload()
file_name = list(uploaded.keys())[0]
file_path = os.path.join(save_dir, file_name)
with open(file_path, "wb") as f:
    f.write(uploaded[file_name])
print(f"File saved at: {file_path}")


In [ ]:
#@title 合成音频（推理）
#@markdown 需要将音频上传到so-vits-svc/raw 文件夹下, 然后设置模型路径、配置文件路径、合成的音频名称

!python inference_main.py -m "logs/44k/G_1600.pth" -c "configs/config.json" -n "君の知らない物語-src.wav" -s kiritan


In [ ]:
#@title 合成音频（推理）修改
#@markdown 需要将音频上传到so-vits-svc/raw 文件夹下, 然后设置模型路径、配置文件路径、合成的音频名称
model_path = "logs/44k/G_2000.pth" #@param {type:"string"}
config_path = "configs/config.json" #@param {type:"string"}
clean_names = "\u98A8\u306B\u306A\u308B.wav" #@param {type:"string"}
spk_list = "kumi" #@param {type:"string"}
trans = "0"  #@param {type:"string"}
#@markdown **聚类音色泄漏控制**

#@markdown 0为完全不使用聚类，1为只使用聚类，通常设置0.5即可
cluster_infer_ratio = "0"  #@param {type:"string"}
slice_db = "-40"  #@param {type:"string"}
wav_format = "flac"  #@param {type:"string"}
wav_output = "/content/so-vits-svc/results/" + clean_names + "_" + trans + "key" + "_" + spk_list + "." + wav_format

!python inference_main.py -m {model_path} -c {config_path} -n "{clean_names}" -t {trans} -s {spk_list}  -cr {cluster_infer_ratio} -sd {slice_db} -wf {wav_format}

#@markdown 如果您不想從此處下載，請取消選中此選項。
download_after_inference = True  #@param {type:"boolean"}

if download_after_inference:
  from google.colab import files
  files.download(wav_output)